Firstly, we install the necessary components for pyspark

In [1]:
!pip install -q findspark
!pip install pyspark
!pip install py4j
!pip install --upgrade py4j

  Using cached py4j-0.10.9.9-py2.py3-none-any.whl.metadata (1.3 kB)
Using cached py4j-0.10.9.9-py2.py3-none-any.whl (203 kB)
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyspark 3.5.4 requires py4j==0.10.9.7, but you have py4j 0.10.9.9 which is incompatible.


import the necessary libraries

In [2]:
import json, requests
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col, array, struct, lit, get_json_object
import json
import findspark

set up variables for the GET URL request

In [3]:
API_KEY = '14gfkN3hD26VRnFO5CXk3WpUTXKETxdSdY4FjMEL'
START_DATE = '2024-12-01'
END_DATE = '2024-12-5'
url = 'https://api.nasa.gov/neo/rest/v1/feed?start_date={}&end_date={}&api_key={}'.format(START_DATE, END_DATE, API_KEY)

save the received json file to 'result' (the first entry is also displayed)

In [10]:
findspark.init()
findspark.find()
response = requests.get(url)
result = response.json()

#example entry from json file
result['near_earth_objects']['2024-12-01'][0]

{'links': {'self': 'http://api.nasa.gov/neo/rest/v1/neo/54065916?api_key=14gfkN3hD26VRnFO5CXk3WpUTXKETxdSdY4FjMEL'},
 'id': '54065916',
 'neo_reference_id': '54065916',
 'name': '(2020 TH6)',
 'nasa_jpl_url': 'https://ssd.jpl.nasa.gov/tools/sbdb_lookup.html#/?sstr=54065916',
 'absolute_magnitude_h': 28.8,
 'estimated_diameter': {'kilometers': {'estimated_diameter_min': 0.0046190746,
   'estimated_diameter_max': 0.0103285648},
  'meters': {'estimated_diameter_min': 4.6190746028,
   'estimated_diameter_max': 10.328564805},
  'miles': {'estimated_diameter_min': 0.002870159,
   'estimated_diameter_max': 0.0064178706},
  'feet': {'estimated_diameter_min': 15.1544447199,
   'estimated_diameter_max': 33.886368555}},
 'is_potentially_hazardous_asteroid': False,
 'close_approach_data': [{'close_approach_date': '2024-12-01',
   'close_approach_date_full': '2024-Dec-01 22:55',
   'epoch_date_close_approach': 1733093700000,
   'relative_velocity': {'kilometers_per_second': '4.5328253058',
    'kil

We intialize a pyspark session

In [5]:
spark= SparkSession \
       .builder \
       .appName("asteroid data") \
       .getOrCreate()

Now we create a loop that extracts the NEO's id, name, and miss distance in miles. Afterwards, it saved to Pyspark dataframe.

In [6]:
neo_list = []
for date, neos in result['near_earth_objects'].items():
    for neo in neos:
        miss_distance_miles = neo['close_approach_data'][0]['miss_distance']['miles']
        neo_list.append((neo['id'], neo['name'], miss_distance_miles))


df = spark.createDataFrame(neo_list, ["id", "name", "miss_distance_miles"])


df.show()

+--------+-----------+-------------------+
|      id|       name|miss_distance_miles|
+--------+-----------+-------------------+
| 3442841|(2008 YS27)| 10813531.994023308|
| 3460325| (2009 KY1)|14046699.5700268952|
| 3656409|(2014 AA53)|17402930.8542236912|
| 3789470| (2017 WL1)| 10202285.614528609|
|54213628| (2021 UQ4)| 32733412.069450866|
|54225710| (2021 WA5)|  2335547.532105324|
|54229471| (2021 XN4)|   4189209.09836505|
|54340322| (2023 BJ7)|15844787.7028767138|
|54354403| (2023 GJ2)| 43384262.804562119|
|54500031| (2024 WB1)| 4553595.0398739572|
|54500315| (2024 WT1)|  5691660.437083179|
|54500320| (2024 WY1)| 7395539.1619332816|
|54501473| (2024 WG5)| 2842485.4873778878|
|54502663|  (2024 XO)| 12504393.271461396|
|54502873| (2024 XD1)| 14284244.348958591|
|54503173|(2024 WA22)| 1348024.2951026172|
|54503203| (2024 XW3)| 3980841.9849891792|
|54503839| (2024 XV4)| 11480895.049953759|
|54506447|(2024 XO15)| 1653294.8244653736|
|54506634|(2024 XB17)|  8552903.355711259|
+--------+-